# Atproto OAuth

This turned out to be a major pain in the butt to get to work with Deno. See [this](https://github.com/bluesky-social/atproto/discussions/3303)

Start by patching `jwk-jose` from https://github.com/bluesky-social/atproto/tree/main/packages/oauth/jwk-jose. Basically copy what's there in atproto repo and use jose from denoland URL. Also, patch this line to include algorithm explicitly (see [this](https://github.com/panva/jose/discussions/648)):

```ts
return (this.#keyObj ||= await importJWK(this.jwk as JWK, "ES256"));
```

In [ ]:
// |export

import {
  errors,
  exportJWK,
  generateKeyPair,
  type GenerateKeyPairOptions,
  type GenerateKeyPairResult,
  importJWK,
  importPKCS8,
  type JWK,
  jwtVerify,
  type JWTVerifyOptions,
  type KeyLike,
  SignJWT,
} from "https://deno.land/x/jose@v5.9.6/index.ts";

import {
  Jwk,
  JwkError,
  jwkValidator,
  JwtCreateError,
  JwtHeader,
  jwtHeaderSchema,
  JwtPayload,
  jwtPayloadSchema,
  JwtVerifyError,
  Key,
  RequiredKey,
  SignedJwt,
  VerifyOptions,
  VerifyResult,
} from "@atproto/jwk";

export function either<T extends string | number | boolean>(
  a?: T,
  b?: T,
): T | undefined {
  if (a != null && b != null && a !== b) {
    throw new TypeError(`Expected "${b}", got "${a}"`);
  }
  return a ?? b ?? undefined;
}
const { JOSEError } = errors;

export type Importable = string | KeyLike | Jwk;

export type { GenerateKeyPairOptions, GenerateKeyPairResult };

export class JoseKey<J extends Jwk = Jwk> extends Key<J> {
  /**
   * Some runtimes (e.g. Bun) require an `alg` second argument to be set when
   * invoking `importJWK`. In order to be compatible with these runtimes, we
   * provide the following method to ensure the `alg` is always set. We also
   * take the opportunity to ensure that the `alg` is compatible with this key.
   */
  protected async getKeyObj(alg: string) {
    if (!this.algorithms.includes(alg)) {
      throw new JwkError(`Key cannot be used with algorithm "${alg}"`);
    }
    try {
      return await importJWK(this.jwk as JWK, alg);
    } catch (cause) {
      throw new JwkError("Failed to import JWK", undefined, { cause });
    }
  }

  async createJwt(header: JwtHeader, payload: JwtPayload): Promise<SignedJwt> {
    try {
      const { kid } = header;
      if (kid && kid !== this.kid) {
        throw new JwtCreateError(
          `Invalid "kid" (${kid}) used to sign with key "${this.kid}"`,
        );
      }

      const { alg } = header;
      if (!alg) {
        throw new JwtCreateError('Missing "alg" in JWT header');
      }

      const keyObj = await this.getKeyObj(alg);
      const jwtBuilder = new SignJWT(payload).setProtectedHeader({
        ...header,
        alg,
        kid: this.kid,
      });

      const signedJwt = await jwtBuilder.sign(keyObj);

      return signedJwt as SignedJwt;
    } catch (cause) {
      if (cause instanceof JOSEError) {
        throw new JwtCreateError(cause.message, cause.code, { cause });
      } else {
        throw JwtCreateError.from(cause);
      }
    }
  }

  async verifyJwt<C extends string = never>(
    token: SignedJwt,
    options?: VerifyOptions<C>,
  ): Promise<VerifyResult<C>> {
    try {
      const result = await jwtVerify(
        token,
        async ({ alg }) => await this.getKeyObj(alg),
        { ...options, algorithms: this.algorithms } as JWTVerifyOptions,
      );

      // @NOTE if all tokens are signed exclusively through createJwt(), then
      // there should be no need to parse the payload and headers here. But
      // since the JWT could have been signed with the same key from somewhere
      // else, let's parse it to ensure the integrity (and type safety) of the
      // data.
      const headerParsed = jwtHeaderSchema.safeParse(result.protectedHeader);
      if (!headerParsed.success) {
        throw new JwtVerifyError("Invalid JWT header", undefined, {
          cause: headerParsed.error,
        });
      }

      const payloadParsed = jwtPayloadSchema.safeParse(result.payload);
      if (!payloadParsed.success) {
        throw new JwtVerifyError("Invalid JWT payload", undefined, {
          cause: payloadParsed.error,
        });
      }

      return {
        protectedHeader: headerParsed.data,
        // "requiredClaims" enforced by jwtVerify()
        payload: payloadParsed.data as RequiredKey<JwtPayload, C>,
      };
    } catch (cause) {
      if (cause instanceof JOSEError) {
        throw new JwtVerifyError(cause.message, cause.code, { cause });
      } else {
        throw JwtVerifyError.from(cause);
      }
    }
  }

  static async generateKeyPair(
    allowedAlgos: readonly string[] = ["ES256"],
    options?: GenerateKeyPairOptions,
  ) {
    if (!allowedAlgos.length) {
      throw new JwkError("No algorithms provided for key generation");
    }

    const errors: unknown[] = [];
    for (const alg of allowedAlgos) {
      try {
        return await generateKeyPair(alg, options);
      } catch (err) {
        errors.push(err);
      }
    }

    throw new JwkError("Failed to generate key pair", undefined, {
      cause: new AggregateError(errors, "None of the algorithms worked"),
    });
  }

  static async generate(
    allowedAlgos: string[] = ["ES256"],
    kid?: string,
    options?: Omit<GenerateKeyPairOptions, "extractable">,
  ) {
    const kp = await this.generateKeyPair(allowedAlgos, {
      ...options,
      extractable: true,
    });
    return this.fromImportable(kp.privateKey, kid);
  }

  static async fromImportable(
    input: Importable,
    kid?: string,
  ): Promise<JoseKey> {
    if (typeof input === "string") {
      // PKCS8
      if (input.startsWith("-----")) {
        // The "alg" is only needed in WebCrypto (NodeJS will be fine)
        return await this.fromPKCS8(input, "", kid);
      }

      // Jwk (string)
      if (input.startsWith("{")) {
        return await this.fromJWK(input, kid);
      }

      throw new JwkError("Invalid input");
    }

    if (typeof input === "object") {
      // Jwk
      if ("kty" in input || "alg" in input) {
        return await this.fromJWK(input, kid);
      }

      // KeyLike
      return await this.fromKeyLike(input, kid);
    }

    throw new JwkError("Invalid input");
  }

  /**
   * @see {@link exportJWK}
   */
  static async fromKeyLike(
    keyLike: KeyLike | Uint8Array,
    kid?: string,
    alg?: string,
  ): Promise<JoseKey> {
    const jwk = await exportJWK(keyLike);
    if (alg) {
      if (!jwk.alg) jwk.alg = alg;
      else if (jwk.alg !== alg) throw new JwkError('Invalid "alg" in JWK');
    }
    // @ts-ignore yolo
    return this.fromJWK(jwk, kid);
  }

  /**
   * @see {@link importPKCS8}
   */
  static async fromPKCS8(
    pem: string,
    alg: string,
    kid?: string,
  ): Promise<JoseKey> {
    const keyLike = await importPKCS8(pem, alg, { extractable: true });
    return this.fromKeyLike(keyLike, kid);
  }

  // deno-lint-ignore require-await
  static async fromJWK(
    input: string | Record<string, unknown>,
    inputKid?: string,
  ): Promise<JoseKey> {
    const jwk = typeof input === "string" ? JSON.parse(input) : input;
    if (!jwk || typeof jwk !== "object") throw new JwkError("Invalid JWK");

    const kid = either(jwk.kid, inputKid);
    const use = jwk.use || "sig";

    return new JoseKey(jwkValidator.parse({ ...jwk, kid, use }));
  }
}

# Recreate dpop-store

This is from https://github.com/bluesky-social/atproto/blob/main/packages/oauth/oauth-client-node/src/node-dpop-store.ts to simplify key storage

In [ ]:
// |export

import { SimpleStore } from "@atproto-labs/simple-store";
import { InternalStateData, Session } from "@atproto/oauth-client";

type ToDpopJwkValue<V extends { dpopKey: Key }> = Omit<V, "dpopKey"> & {
  dpopJwk: Jwk;
};

/**
 * Utility function that allows to simplify the store interface by exposing a
 * JWK (JSON) instead of a Key instance.
 */
export function toDpopKeyStore<K extends string, V extends { dpopKey: Key }>(
  store: SimpleStore<K, ToDpopJwkValue<V>>,
): SimpleStore<K, V> {
  return {
    async set(sub: K, { dpopKey, ...data }: V) {
      const dpopJwk = dpopKey.privateJwk;
      if (!dpopJwk) throw new Error("Private DPoP JWK is missing.");

      await store.set(sub, { ...data, dpopJwk });
    },

    async get(sub: K) {
      const result = await store.get(sub);
      if (!result) return undefined;

      const { dpopJwk, ...data } = result;
      const dpopKey = await JoseKey.fromJWK(dpopJwk);
      return { ...data, dpopKey } as unknown as V;
    },

    del: store.del.bind(store),
    clear: store.clear?.bind(store),
  };
}

export type NodeSavedState = ToDpopJwkValue<InternalStateData>;
export type NodeSavedStateStore = SimpleStore<string, NodeSavedState>;

export type NodeSavedSession = ToDpopJwkValue<Session>;
export type NodeSavedSessionStore = SimpleStore<string, NodeSavedSession>;